In [2]:
%pip install yahooquery

  Attempting uninstall: lxml
    Found existing installation: lxml 5.3.0
    Uninstalling lxml-5.3.0:
      Successfully uninstalled lxml-5.3.0
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\gianm\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\~xml\\etree.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\Users\gianm\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
import yfinance as yf
import pandas as pd

symbols = [ 'VFC',  'INLX', 'INVO', 'IONM']
sectorDict = {}
snpClosing = pd.DataFrame()  

for sector in symbols:
    snpSect = yf.download(tickers=sector,start="2019-01-01" , end="2021-12-31")
    snpClosingSymbol = snpSect['Close'].reset_index()
    snpClosingSymbol['Date'] = snpClosingSymbol['Date'].dt.strftime('%Y-%m-%d')
    snpClosingSymbol[sector] = snpClosingSymbol[sector].round(2)
    #snpClosingSymbol.drop(columns=['Close'], inplace=True)
    snpClosingSymbol.set_index('Date', inplace=True)
    if snpClosing.empty:
        snpClosing = snpClosingSymbol
    else:
        snpClosing = pd.merge(snpClosing, snpClosingSymbol, on='Date', how='outer')

#snpClosing.set_index('Date', inplace=True)
snpClosing.to_csv("snp_sectors.csv")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
